# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

### Streaming tick data

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 2, 5, 18, 19, 26, 776912, tzinfo=datetime.timezone.utc), bid=1.14135, bidSize=5000000, ask=1.14138, askSize=1000000, prevBid=1.14136, prevBidSize=500000, high=1.14405, low=1.14015, close=1.1438, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.141365

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,1000000,1.14138,1.14139,2500000,1.14405,1.14015,1.1438
USDJPY,2000000,109.875,109.877,7000000,110.045,109.775,109.9
GBPUSD,1000000,1.29558,1.29564,2000000,1.30525,1.29245,1.3036
USDCHF,500000,0.99982,0.99984,1000000,1.00205,0.9976,0.9979
USDCAD,3000000,1.3137,1.31373,2000000,1.31535,1.31025,1.311
AUDUSD,1000000,0.72369,0.72371,4500000,0.7265,0.71935,0.7226


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [8]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2019, 2, 5, 18, 19, 55, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(), priceBid=1.14138, priceAsk=1.14139, sizeBid=1000000, sizeAsk=2500000)

In [9]:
ib.disconnect()